In [ ]:
import cr_autophagy as cra

# Imports of general-purpose python libraries
from pathlib import Path

# output_path = cra.get_last_output_path()
output_path = Path("out/autophagy/2023-12-12-T00-34-23")
simulation_settings = cra.get_simulation_settings(output_path)
max_iter = max(cra.get_all_iterations(output_path))

In [ ]:
import cc3d
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass


def calcualte_3d_connected_components(mask):
    labels = cc3d.connected_components(mask)
    cluster_identifiers = np.unique(labels)
    n_clusters = len(cluster_identifiers)
    cluster_sizes = np.array([len(labels[labels==i]) for i in cluster_identifiers])
    return n_clusters, labels, cluster_identifiers, cluster_sizes


def determine_optimal_thresh(
        output_path,
        iteration,
        discretization_factor,
        bw_method=0.45,
        dthresh=0.01,
    ):
    # Define starting values
    n_clusters = 2
    threshold = dthresh

    # As long as the cargo is still recognized as one cluster, we are lowering
    while n_clusters == 2:
        threshold += dthresh

        density_cargo, _ = cra.calculate_kernel_densities(
            output_path,
            iteration,
            discretization_factor,
            bw_method
        )
        mask_cargo, _ = cra._calculate_mask(density_cargo, threshold)

        n_clusters, _, _, _ = calcualte_3d_connected_components(mask_cargo)

    # We return the last known value where bw_method was successful
    return threshold - dthresh


@dataclass
class ClusterResult:
    n_clusters: int
    cluster_sizes: np.ndarray
    positions_r11: np.ndarray
    cargo_position: np.ndarray
    distances_cargo: np.ndarray


def calculate_cargo_r11_cluster_distances(mask_r11, mask_cargo, domain_size) -> ClusterResult:
    _, labels_cargo, identifiers_cargo, _ = calcualte_3d_connected_components(mask_cargo)
    n_clusters, labels_r11, identifiers_r11, cluster_sizes = calcualte_3d_connected_components(mask_r11)

    _helper_x = domain_size/np.array([*labels_cargo.shape])
    _helper_y = np.product(_helper_x)**(1/3)
    to_coordinate = lambda x: x*_helper_x
    to_coordinate_single = lambda x: x*_helper_y

    # Transform cluster_sizes to coordinates
    cluster_sizes = to_coordinate_single(cluster_sizes)

    positions_cargo = np.array([])
    for ident in identifiers_cargo[1:]:
        indices = np.argwhere(labels_cargo==ident)
        middle = to_coordinate(np.average(indices, axis=0))
        distances_cargo = np.sum((to_coordinate(indices) - middle)**2, axis=1)**0.5
        positions_cargo = np.vstack([*positions_cargo, middle])

    # Make sure that we only have one cargo position left over
    if positions_cargo.shape[0] != 1:
        return

    cargo_position = positions_cargo[0]
    if cargo_position.shape != (3,):
        return

    # Now gather positions of R11 clusters
    positions_r11 = np.array([])
    for ident in identifiers_r11[1:]:
        indices = np.argwhere(labels_r11==ident)
        middle = np.average(indices, axis=0)*domain_size/np.array([*labels_r11.shape])
        positions_r11 = np.vstack([*positions_r11, middle])

    # Make sure that we really have some R11 clusters
    if positions_r11.shape[0] == 0:
        return

    return ClusterResult(
        n_clusters-1,
        cluster_sizes[1:],
        positions_r11[1:],
        cargo_position,
        distances_cargo
    )


In [ ]:
import importlib
importlib.reload(cra)


def get_clusters(output_path, iteration, *args):
    simulation_settings = cra.get_simulation_settings(output_path)
    domain_size = simulation_settings.domain_size

    threshold = determine_optimal_thresh(
        output_path,
        iteration,
        *args,
    )

    density_cargo, density_r11 = cra.calculate_kernel_densities(
        output_path,
        iteration,
        *args,
    )

    mask_r11,   _ = cra._calculate_mask(density_r11,   threshold)
    mask_cargo, _ = cra._calculate_mask(density_cargo, threshold)

    return calculate_cargo_r11_cluster_distances(mask_r11, mask_cargo, domain_size)


def plot_cluster_distribution(output_path, iteration, discretization_factor, bw_method):
    clrs = get_clusters(output_path, iteration, discretization_factor, bw_method)

    # Calculate percentiles for plotting
    percentiles = []
    for perc in [70, 80, 90]:
        percentiles.append(np.percentile(clrs.distances_cargo, perc))

    distances = np.array([
        np.sum((x-clrs.cargo_position)**2)**0.5 for x in clrs.positions_r11
    ])

    fig, ax = plt.subplots()
    # ax.hist(labels[labels!=np.argmax(cluster_sizes)])
    ax.set_ylabel("Cluster Volume")
    ax.set_xlabel("Distance to Cargo")
    ax.set_title(f"{clrs.n_clusters-1} Cluster" + (clrs.n_clusters-1>1)*"s")
    ax.scatter(distances, clrs.cluster_sizes[1:], color="#A23302")

    last_percentile = 0
    max_percentile = np.max(percentiles)
    for p in percentiles:
        ax.axvspan(last_percentile, p, alpha=1-p/max_percentile, color="#072853")
        last_percentile = p
    return fig

discretization_factor = 0.5
bw_method = 0.12
plot_cluster_distribution(output_path, max_iter, discretization_factor, bw_method)

In [ ]:
fig = cra.save_kernel_density(
    output_path,
    max_iter,
    overwrite=True,
    threshold=0.001,
    discretization_factor=0.5,
    bw_method=0.15
)

In [ ]:
# cra.save_all_kernel_density(output_path, threshold=0.6, overwrite=True, bw_method=0.4, discretization_factor=0.5)
# 
# bashcmd = f"ffmpeg -v quiet -stats -y -r 30 -f image2 -pattern_type glob -i '{output_path}/kernel_density/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/kernel_density_movie.mp4"
# os.system(bashcmd)